# Pipeline 1: Collecting Twitter Data

In [35]:
# Importing libraries
import sys
#!{sys.executable} -m pip install <library>
import pandas as pd
import numpy as np
from timeit import timeit
import boto3
import s3fs
import datetime as dt
import os
import tweepy


In [24]:
# SECURITY: Save keys in a separate file and run it in this file using the code below:

# %run ./<filename>.ipynb


# Authentication Credentials **Remove for security**
consumer_key = "7Q6AcYjJSJ0rFCofWE9X5km0g"
consumer_secret = "1LiC3fdtRACZckDoAzTmNGmZXfwtxXapfnwiEcXPML27SbJJ2d"
access_token = "1495039738323124226-dgQZUL9iCP1ksx0tptql4ZwymHgCdG"
access_token_secret = "ocjHQL0VuWwAMjrSL6kbvx470l2ru011bxb8ete8w8Osi"

In [25]:
# Authenticating Access

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [26]:
# Enabling Twitter navigation: Searching by Key Words "test"

cursor = tweepy.Cursor(api.search_tweets, q="jews", tweet_mode="extended").items(1)

In [27]:
# Printing "Test" Results

for i in cursor:
    print(i.full_text)

@ellie_pool @Mrs_Right_A @ValValpuriam She organized an illegal occupation of Ottawa.  She has hurt the locals. She has hurt her followers.  She has hurt our economy.  She has hurt our reputation. The 306 million dollar lawsuit isn't for fun.  She deserves what she gets.  The Jews did not.


In [36]:
# Setting up Extraction method for pipeline data feeding

number_of_tweets = 200
tweets = []
likes = []
time = []

for i in tweepy.Cursor(api.search_tweets, q="jews", tweet_mode="extended").items(number_of_tweets):
    tweets.append(i.full_text)
    likes.append(i.favorite_count)
    time.append(i.created_at)
    

In [38]:
# Array to Dataframe

df = pd.DataFrame({'tweets':tweets, 'likes':likes, 'time':time})

df

,tweets,likes,time
0,@MrDessert_ @zinda_tilismaat Messi I guess he'...,0,2022-02-19 18:51:05+00:00
1,"6 million people didn't choose ""move on"" not s...",0,2022-02-19 18:51:01+00:00
2,Israel has conveniently dubbed Jews who do not...,0,2022-02-19 18:50:55+00:00
3,RT @InsightGL: -Please don't take me wrong but...,0,2022-02-19 18:50:50+00:00
4,RT @AuschwitzMuseum: 19 February 1912 | A Dutc...,0,2022-02-19 18:50:46+00:00
...,...,...,...
195,RT @AuschwitzMuseum: 19 February 1912 | A Dutc...,0,2022-02-19 18:37:15+00:00
196,RT @AndreaZanardo8: A group of Jewish teenager...,0,2022-02-19 18:37:15+00:00
197,RT @AuschwitzMuseum: 18 February 1943 | A tran...,0,2022-02-19 18:37:14+00:00
198,@brbuss @Electshelby @GovernorLittle Something...,0,2022-02-19 18:37:12+00:00


# EDA

In [40]:
# Data Pre-Processing -- Refinery Flow: Removing Re-Tweets

df = df[~df.tweets.str.contains("RT")].reset_index(drop=True)

df

,tweets,likes,time
0,@MrDessert_ @zinda_tilismaat Messi I guess he'...,0,2022-02-19 18:51:05+00:00
1,"6 million people didn't choose ""move on"" not s...",0,2022-02-19 18:51:01+00:00
2,Israel has conveniently dubbed Jews who do not...,0,2022-02-19 18:50:55+00:00
3,@AOC You are an admitted antisemite and you re...,0,2022-02-19 18:50:46+00:00
4,@EpshtainItay Less than 5% of WB has settlers\...,0,2022-02-19 18:50:41+00:00
...,...,...,...
80,No sympathy for the drivers hiding their ident...,0,2022-02-19 18:37:32+00:00
81,@RashidaTlaib Sounds like we should stop givin...,0,2022-02-19 18:37:22+00:00
82,#Epstein Tucker Carlson #MarjorieTaylorGreene ...,0,2022-02-19 18:37:16+00:00
83,@brbuss @Electshelby @GovernorLittle Something...,0,2022-02-19 18:37:12+00:00


In [42]:
# Most Liked Tweets
mostliked = df.loc[df.likes.nlargest(5).index]

mostliked

,tweets,likes,time
67,-Please don't take me wrong but\n-6 million #J...,28,2022-02-19 18:39:52+00:00
64,@lambdawon @Thedukeistheman @garden_liliths @h...,2,2022-02-19 18:40:44+00:00
25,diego won!!! a win for the Jews,1,2022-02-19 18:47:19+00:00
44,@LindaWesson @alexplitsas Am always amazed how...,1,2022-02-19 18:43:42+00:00
45,@lambdawon @MuddyPenguins @Thedukeistheman @ga...,1,2022-02-19 18:43:41+00:00


# Connecting to S3 and Importing Data

In [44]:
%%time
# creating a variable that stores date to be append on the file to be archive
today = dt.datetime.today()
date = today.strftime("%Y%m%d")
#connect and load the historical data file to s3

s3pathName = 's3://tweetsdatatest/'
FileNameFullPath = 'tweets_test1' 
filenames3 = "%s%s%s.csv"%(s3pathName,FileNameFullPath, date) #name of the filepath and csv file

df.to_csv(filenames3, header=True, line_terminator='\n') 

#print success message
print("Successfull uploaded file to location:"+str(filenames3))

Successfull uploaded file to location:s3://tweetsdatatest/tweets_test120220219.csv
Wall time: 630 ms
